<a href="https://colab.research.google.com/github/Manith-Ratnayake/UCI_machine_learning/blob/main/CIFAR-10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for training and testing data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Define the Neural Network
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the network and optimizer
net = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training loop
def train_model(num_epochs=10):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = net(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Optimize

            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 100:.3f}")
                running_loss = 0.0

    print('Finished Training')

# Model evaluation
def evaluate_model():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy on test images: {100 * correct / total:.2f}%')

# Visualizing some test images and predictions
def visualize_test_images():
    dataiter = iter(testloader)
    images, labels = dataiter.next()

    # Print images and labels
    def imshow(img):
        img = img / 2 + 0.5  # Unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.show()

    imshow(torchvision.utils.make_grid(images))
    print('GroundTruth:', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

    # Predict using the model
    images = images.to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)

    print('Predicted:', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(4)))

# Running the training and evaluation
if __name__ == "__main__":
    train_model(num_epochs=10)
    evaluate_model()
    visualize_test_images()


100%|██████████| 170498071/170498071 [00:03<00:00, 42731757.11it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[Epoch 1, Batch 100] Loss: 1.826
[Epoch 1, Batch 200] Loss: 1.476
[Epoch 1, Batch 300] Loss: 1.355
[Epoch 1, Batch 400] Loss: 1.294
[Epoch 1, Batch 500] Loss: 1.220
[Epoch 1, Batch 600] Loss: 1.172
[Epoch 1, Batch 700] Loss: 1.127
[Epoch 2, Batch 100] Loss: 1.000
[Epoch 2, Batch 200] Loss: 0.992
[Epoch 2, Batch 300] Loss: 0.974
[Epoch 2, Batch 400] Loss: 0.940
[Epoch 2, Batch 500] Loss: 0.936
[Epoch 2, Batch 600] Loss: 0.910
[Epoch 2, Batch 700] Loss: 0.880
[Epoch 3, Batch 100] Loss: 0.787
[Epoch 3, Batch 200] Loss: 0.793
[Epoch 3, Batch 300] Loss: 0.770
[Epoch 3, Batch 400] Loss: 0.777
[Epoch 3, Batch 500] Loss: 0.770
[Epoch 3, Batch 600] Loss: 0.779
[Epoch 3, Batch 700] Loss: 0.735
[Epoch 4, Batch 100] Loss: 0.613
[Epoch 4, Batch 200] Loss: 0.659
[Epoch 4, Batch 300] Loss: 0.659
[Epoch 4, Batch 400] Loss: 0.640
[Epoch 4, Batch 500] Loss: 0.636
[Epoch 4, Batch 600] Loss: 0.643
[Epoch 4, Batch 700]

AttributeError: '_SingleProcessDataLoaderIter' object has no attribute 'next'